# 範例 : (Kaggle)房價預測

# [教學目標]
- 以下用房價預測資料, 觀察群聚編碼的效果

# [範例重點]
- 了解群聚編碼的寫作方式(In[3], Out[3])
- 觀察群聚編碼, 搭配線性迴歸以及隨機森林分別有什麼影響 (In[6]~In[9], Out[6]~Out[9])

In [1]:
# 請先確認您的 sklearn 版本是否相同, 如果版本不是 0.21.1 版, 有可能跑出與本範例不同的結果
import sklearn
sklearn.__version__
# pip install --upgrade sklearn==0.21.1
# conda install -c anaconda scikit-learn

'0.20.1'

In [2]:
# 做完特徵工程前的所有準備
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'house_train.csv.gz')

train_Y = np.log1p(df['SalePrice'])
df = df.drop(['Id', 'SalePrice'] , axis=1)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
# 生活總面積(GrLivArea)對販售條件(SaleCondition)做群聚編碼
# 寫法類似均值編碼, 只是對另一個特徵, 而非目標值
df['SaleCondition'] = df['SaleCondition'].fillna('None') # 原始資料中SaleCondition並沒有nan值
mean_df = df.groupby(['SaleCondition'])['GrLivArea'].mean().reset_index()
mode_df = df.groupby(['SaleCondition'])['GrLivArea'].apply(lambda x: x.mode()[0]).reset_index() # 眾數
median_df = df.groupby(['SaleCondition'])['GrLivArea'].median().reset_index()
max_df = df.groupby(['SaleCondition'])['GrLivArea'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['SaleCondition'])
temp = pd.merge(temp, median_df, how='left', on=['SaleCondition'])
temp = pd.merge(temp, max_df, how='left', on=['SaleCondition'])
temp.columns = ['SaleCondition', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']
temp

,SaleCondition,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,Abnorml,1436.128713,864,1302.0,4476
1,AdjLand,1112.500000,980,1143.0,1184
2,Alloca,1701.750000,1535,1439.5,3194
3,Family,1480.950000,948,1390.5,2526
4,Normal,1492.968280,864,1456.0,4316
5,Partial,1795.696000,1456,1646.0,5642


In [4]:
temp.shape

(6, 5)

In [5]:
df = pd.merge(df, temp, how='left', on=['SaleCondition'])
df = df.drop(['SaleCondition'] , axis=1)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,NaN,NaN,0,2,2008,WD,1492.968280,864,1456.0,4316
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,NaN,NaN,0,5,2007,WD,1492.968280,864,1456.0,4316
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,NaN,NaN,0,9,2008,WD,1492.968280,864,1456.0,4316
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,NaN,NaN,0,2,2006,WD,1436.128713,864,1302.0,4476
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,NaN,NaN,0,12,2008,WD,1492.968280,864,1456.0,4316


In [6]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

40 Numeric Features : ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max']



,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,Area_Sale_Mean,Area_Sale_Mode,Area_Sale_Median,Area_Sale_Max
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,0,2,2008,1492.968280,864,1456.0,4316
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,0,5,2007,1492.968280,864,1456.0,4316
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,0,9,2008,1492.968280,864,1456.0,4316
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,0,2,2006,1436.128713,864,1302.0,4476
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,0,12,2008,1492.968280,864,1456.0,4316


In [7]:
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['Area_Sale_Mean', 'Area_Sale_Mode', 'Area_Sale_Median', 'Area_Sale_Max'] , axis=1)

# 原始特徵 + 線性迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LinearRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()
# 0.8466510874327978

0.8499683837163878

In [8]:
# 新特徵 + 線性迴歸 : 有些微改善
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()
# 0.8492735339540589

0.8491312626992871

In [9]:
# 原始特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df_minus)
estimator = RandomForestRegressor()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()
# 0.8485866392278265

0.8495026694771456

In [10]:
# 新特徵 + 隨機森林
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()
# 0.8461117221094827

0.8494925019537796

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [4]:
df = pd.DataFrame({'key1':list('aabba'),'key2':['one','two','one','two','one'],
   'data1':np.random.randn(5),'data2':np.random.randn(5)})

In [5]:
df

,key1,key2,data1,data2
0,a,one,0.907407,-1.991005
1,a,two,-0.855700,0.406135
2,b,one,-1.650125,0.357997
3,b,two,0.803832,-0.448800
4,a,one,-1.013795,-1.835296


In [8]:
grouped = df["data1"].groupby(df["key1"])

In [11]:
grouped.mean()

key1
a   -0.320696
b   -0.423147
Name: data1, dtype: float64

In [12]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()

In [13]:
means

key1  key2
a     one    -0.053194
      two    -0.855700
b     one    -1.650125
      two     0.803832
Name: data1, dtype: float64

In [15]:
means.unstack()

key2,one,two
key1,,
a,-0.053194,-0.855700
b,-1.650125,0.803832


In [16]:
df.groupby("key1").mean()

,data1,data2
key1,,
a,-0.320696,-1.140055
b,-0.423147,-0.045402


In [19]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    one  -0.053194 -1.913150
     two  -0.855700  0.406135
b    one  -1.650125  0.357997
     two   0.803832 -0.448800

In [20]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [21]:
pieces = dict(list(df.groupby("key1")))

In [25]:
pieces["b"]

,key1,key2,data1,data2
2,b,one,-1.650125,0.357997
3,b,two,0.803832,-0.448800


In [27]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [28]:
groupbyed = df.groupby(df.dtypes, axis=1)

In [30]:
a = dict(list(grouped))

In [37]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.913150
     two   0.406135
b    one   0.357997
     two  -0.448800

In [40]:
people = pd.DataFrame(np.random.randn(5,5),columns=['a','b','c','d','e'],index=['Joe','Steve','Wes','Jim','Travis'])

In [41]:
people

,a,b,c,d,e
Joe,-0.872124,-1.250617,-0.157484,1.278039,0.568637
Steve,-2.223795,-0.989171,-0.717527,-0.716188,1.086587
Wes,-1.965756,0.531654,-0.986664,-0.762516,0.339328
Jim,2.239878,-0.313902,0.389842,-0.204107,0.379267
Travis,0.070850,-0.095701,-1.053498,1.070625,0.239016


In [42]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.598003,-1.032865,-0.754306,0.311416,1.287232
5,-2.223795,-0.989171,-0.717527,-0.716188,1.086587
6,0.070850,-0.095701,-1.053498,1.070625,0.239016


In [43]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],names=['city','tenor'])

In [44]:
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['city', 'tenor'])

In [46]:
hier_df= pd.DataFrame(np.random.randn(4,5),columns=columns)

In [47]:
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.720206 -1.199891  0.302750  0.224565 -0.593069
1      0.684902  0.239550  0.194663 -0.582893 -1.245796
2     -1.294726  0.661159 -1.599405  0.597966  1.062499
3     -0.128362  0.385297  0.666548 -1.226907 -1.901324

In [48]:
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['city', 'tenor'])

In [53]:
hier_df.groupby(level='city',axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [54]:
df = pd.DataFrame({'key1':list('aabba'),'key2':['one','two','one','two','one'],
'data1':np.random.randn(5),'data2':np.random.randn(5)})

In [55]:
df

,key1,key2,data1,data2
0,a,one,1.140755,0.753607
1,a,two,0.758328,-0.164538
2,b,one,1.466347,0.999790
3,b,two,0.033172,0.349967
4,a,one,-0.414287,0.233107


In [61]:
grouped = df.groupby('key1')

In [62]:
grouped.quantile(0.9)

0.9,data1,data2
key1,,
a,1.064270,0.649507
b,1.323029,0.934808


In [63]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [64]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.555042,0.918146
b,1.433175,0.649823


In [65]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.494932  0.810291 -0.414287  0.172020  0.758328  0.949542   
b      2.0  0.749759  1.013408  0.033172  0.391466  0.749759  1.108053   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.140755   3.0  0.274059  0.460441 -0.164538  0.034284  0.233107   
b     1.466347   2.0  0.674878  0.459494  0.349967  0.512422  0.674878   

                          
           75%       max  
key1                      
a     0.493357  0.753607  
b     0.837334  0.999790

In [66]:
df

,key1,key2,data1,data2
0,a,one,1.140755,0.753607
1,a,two,0.758328,-0.164538
2,b,one,1.466347,0.999790
3,b,two,0.033172,0.349967
4,a,one,-0.414287,0.233107


In [67]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')

In [68]:
k1_means

,mean_data1,mean_data2
key1,,
a,0.494932,0.274059
b,0.749759,0.674878


In [69]:
pd.merge(df,k1_means,left_on = 'key1',right_index = True)

,key1,key2,data1,data2,mean_data1,mean_data2
0,a,one,1.140755,0.753607,0.494932,0.274059
1,a,two,0.758328,-0.164538,0.494932,0.274059
4,a,one,-0.414287,0.233107,0.494932,0.274059
2,b,one,1.466347,0.999790,0.749759,0.674878
3,b,two,0.033172,0.349967,0.749759,0.674878


In [70]:
df.groupby("key1").transform(np.mean)

,data1,data2
0,0.494932,0.274059
1,0.494932,0.274059
2,0.749759,0.674878
3,0.749759,0.674878
4,0.494932,0.274059


In [ ]:
def top(df, n=5, co)